# MN'S WEEK 3 ASSIGNMENT, parts 1, 2, & 3

### Part 1: Create a notebook, scrape a wikipedia page, transform the data into a pandas dataframe.

#### Download the dependencies.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Download the dataset.

In [2]:
toronto_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

#### Explore the dataset.

In [3]:
toronto_data = toronto_data[0]
toronto_data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Ignore "Not Assigned" boroughs.

In [4]:
toronto_data2 = toronto_data[toronto_data['Borough'] != 'Not assigned'].reset_index(drop=True)
                      
toronto_data2['Neighbourhood'] = toronto_data2['Neighbourhood']+'@'+toronto_data2['Borough']
toronto_data2['Neighbourhood'] = toronto_data2['Neighbourhood'].apply(lambda x: x.split('@')[1] if x.split('@')[0] == 'Not assigned' else x.split('@')[0])
                      
toronto_data2

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Aggregate neighborhoods with the same postal code.

In [5]:
gb = toronto_data2.groupby(by='Postcode', as_index=False)

def join_nbhs(x):
    return ', '.join(x)

toronto_data2 = gb.aggregate(join_nbhs)
toronto_data2.head()

,Postcode,Borough,Neighbourhood
0,M1B,"Scarborough, Scarborough","Rouge, Malvern"
1,M1C,"Scarborough, Scarborough, Scarborough","Highland Creek, Rouge Hill, Port Union"
2,M1E,"Scarborough, Scarborough, Scarborough","Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Print number of rows.

In [6]:
print ('Number of rows: {}'.format(toronto_data2.shape[0]))

Number of rows: 103


### Part 2: Add coordinates to the dataframe.

#### Download the coordinates.

In [10]:
toronto_coord = pd.read_csv('http://cocl.us/Geospatial_data')
toronto_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Rename the Postcode column for merging.

In [13]:
toronto_coord.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
toronto_coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge the two tables and rename the Postcode column.

In [14]:
toronto_data3 = toronto_data2.merge(toronto_coord, on='Postcode')
toronto_data3.rename(columns={'Postcode': 'PostalCode'}, inplace=True)
toronto_data3.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,"Scarborough, Scarborough","Rouge, Malvern",43.806686,-79.194353
1,M1C,"Scarborough, Scarborough, Scarborough","Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,"Scarborough, Scarborough, Scarborough","Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Part 3: Generate maps to visualize the neighborhoods. 

#### Look at the unique values for Boroughs.

In [15]:
toronto_data3['Borough'].unique()

array(['Scarborough, Scarborough', 'Scarborough, Scarborough, Scarborough',
       'Scarborough', 'Scarborough, Scarborough, Scarborough, Scarborough',
       'North York', 'North York, North York, North York',
       'North York, North York', 'East York, East York', 'East York',
       'East Toronto', 'East Toronto, East Toronto', 'Central Toronto',
       'Central Toronto, Central Toronto',
       'Central Toronto, Central Toronto, Central Toronto, Central Toronto, Central Toronto',
       'Downtown Toronto', 'Downtown Toronto, Downtown Toronto',
       'Downtown Toronto, Downtown Toronto, Downtown Toronto',
       'Central Toronto, Central Toronto, Central Toronto',
       'Downtown Toronto, Downtown Toronto, Downtown Toronto, Downtown Toronto, Downtown Toronto, Downtown Toronto, Downtown Toronto',
       'York', 'West Toronto, West Toronto',
       'West Toronto, West Toronto, West Toronto',
       'York, York, York, York', 'York, York', "Queen's Park",
       'Mississauga', 'Etobi

#### Eliminate all Borough values except for those that contain "York."

In [17]:
toronto_data4 = toronto_data3[toronto_data3['Borough'].str.match('(.)*(York)')].reset_index(drop=True)
toronto_data4

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,"North York, North York, North York","Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,"North York, North York","Silver Hills, York Mills",43.757490,-79.374714
4,M2M,"North York, North York","Newtonbrook, Willowdale",43.789053,-79.408493
5,M2N,North York,Willowdale South,43.770120,-79.408493
6,M2P,North York,York Mills West,43.752758,-79.400049
7,M2R,North York,Willowdale West,43.782736,-79.442259
8,M3A,North York,Parkwoods,43.753259,-79.329656
9,M3B,North York,Don Mills North,43.745906,-79.352188


#### Find Toronto's coordinates.

In [18]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map with markers for Boroughs containing "York."

In [28]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=112)

for lat, lng, borough, neighborhood in zip(toronto_data4['Latitude'], toronto_data4['Longitude'], toronto_data4['Borough'], toronto_data4['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto